In [1]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
from math import pi, sin, cos
from datetime import datetime
from holidays_es import get_provinces, Province

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

2022-08-29 17:00:51.757427: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-29 17:00:51.757492: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Importing data

In [2]:
df = pd.read_json("../raw_data/Bicimad_Estacions_201808.json",encoding_errors='ignore',lines=True)
df

,_id,stations
0,2018-08-01T00:58:32.872907,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
1,2018-08-01T01:58:33.570038,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
2,2018-08-01T02:58:36.692852,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
3,2018-08-01T03:58:37.185141,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
4,2018-08-01T04:58:40.413655,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
...,...,...
750,2018-08-31T19:20:59.506759,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
751,2018-08-31T20:21:00.259835,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
752,2018-08-31T21:20:59.808767,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
753,2018-08-31T22:21:03.580436,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."


## Adding columns

In [3]:
%%time
df["time"] = "time"
df["date"] = "date"
for i in range(len(df['_id'])):
    item=df['_id'][i]
    groups= item.split("T")
    date = groups[0]
    raw_time = groups[1].split(":")
    time =':'.join(raw_time[:2])
    df["time"][i] = time
    df["date"][i] = date
df.head(3)

CPU times: user 384 ms, sys: 13.6 ms, total: 398 ms
Wall time: 376 ms


,_id,stations,time,date
0,2018-08-01T00:58:32.872907,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",00:58,2018-08-01
1,2018-08-01T01:58:33.570038,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",01:58,2018-08-01
2,2018-08-01T02:58:36.692852,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",02:58,2018-08-01


In [4]:
df['time'] = df.apply(lambda x:datetime.strptime(x['time'], '%H:%M').hour, axis=1)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')


In [5]:
holidays=[]
for year in [2018,2019,2020,2021]:
    temp = Province(name="madrid", year=year).holidays()
    for day in range(len(temp['national_holidays'])):
        holidays.append(temp['national_holidays'][day])
    for day in range(len(temp['regional_holidays'])):
        holidays.append(temp['regional_holidays'][day])
    for day in range(len(temp['local_holidays'])):
        holidays.append(temp['local_holidays'][day])

In [6]:
df_holidays=pd.DataFrame(holidays, columns=['date'])
df_holidays['date'] = pd.to_datetime(df_holidays['date'], format='%Y-%m-%d')
list_holidays=list(df_holidays['date'])
df['holidays']=df['date'].isin(list_holidays)

In [7]:
set(df.holidays)

{False, True}

# THIS TAKES A LOT TO RUN

In [8]:
%%time
temp=pd.DataFrame()
for i in range(755):
    new_dict= df['stations'][i][0]
    new_dict['id']=df['_id'][i]
    new_dict['time']=df['time'][i]
    new_dict['date']=df['date'][i]
    new_dict['holidays']=df['holidays'][i]
    temp = temp.append(new_dict, ignore_index=True)


CPU times: user 3.36 s, sys: 0 ns, total: 3.36 s
Wall time: 3.34 s


In [9]:
temp.head()

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,id,time,date,holidays
0,1.0,Puerta del Sol A,0.0,2.0,24.0,11.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,11.0,2018-08-01T00:58:32.872907,0.0,2018-08-01,0.0
1,1.0,Puerta del Sol A,0.0,0.0,24.0,18.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,4.0,2018-08-01T01:58:33.570038,1.0,2018-08-01,0.0
2,1.0,Puerta del Sol A,0.0,2.0,24.0,13.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,10.0,2018-08-01T02:58:36.692852,2.0,2018-08-01,0.0
3,1.0,Puerta del Sol A,0.0,0.0,24.0,17.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,4.0,2018-08-01T03:58:37.185141,3.0,2018-08-01,0.0
4,1.0,Puerta del Sol A,0.0,0.0,24.0,22.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,0.0,2018-08-01T04:58:40.413655,4.0,2018-08-01,0.0


In [10]:
%%time

temp['weekday'] = temp.apply(lambda x: x['date'].weekday(), axis=1)
temp['year'] = temp.apply(lambda x:x['date'].year, axis=1)
temp['month'] = temp.apply(lambda x: x['date'].month, axis=1)

CPU times: user 42.7 ms, sys: 0 ns, total: 42.7 ms
Wall time: 40 ms


In [11]:
temp.head(2)

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,id,time,date,holidays,weekday,year,month
0,1.0,Puerta del Sol A,0.0,2.0,24.0,11.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,11.0,2018-08-01T00:58:32.872907,0.0,2018-08-01,0.0,2,2018,8
1,1.0,Puerta del Sol A,0.0,0.0,24.0,18.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,4.0,2018-08-01T01:58:33.570038,1.0,2018-08-01,0.0,2,2018,8


### Encoding circular time
temp['hour_sin'] = temp.apply(lambda x: sin(x['hour'] / 24.0 * 2 * pi), axis=1)
temp['hour_cos'] = temp.apply(lambda x: cos(x['hour'] / 24.0 * 2 * pi), axis=1)
temp['weekday_sin'] = temp.apply(lambda x: sin(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['weekday_cos'] = temp.apply(lambda x: cos(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['month_sin'] = temp.apply(lambda x: sin(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['month_cos'] = temp.apply(lambda x:cos(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['season_sin'] = temp.apply(lambda x: sin(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)
temp['season_cos'] = temp.apply(lambda x: cos(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)

In [12]:
temp['hour_sin'] = temp.apply(lambda x: sin(x['time'] / 24.0 * 2 * pi), axis=1)
temp['hour_cos'] = temp.apply(lambda x: cos(x['time'] / 24.0 * 2 * pi), axis=1)
temp['weekday_sin'] = temp.apply(lambda x: sin(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['weekday_cos'] = temp.apply(lambda x: cos(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['month_sin'] = temp.apply(lambda x: sin(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['month_cos'] = temp.apply(lambda x:cos(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)

In [13]:
temp.head()

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,...,holidays,weekday,year,month,hour_sin,hour_cos,weekday_sin,weekday_cos,month_sin,month_cos
0,1.0,Puerta del Sol A,0.0,2.0,24.0,11.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,0.0,2,2018,8,0.000000,1.000000,0.974928,-0.222521,1.0,6.123234e-17
1,1.0,Puerta del Sol A,0.0,0.0,24.0,18.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,0.0,2,2018,8,0.258819,0.965926,0.974928,-0.222521,1.0,6.123234e-17
2,1.0,Puerta del Sol A,0.0,2.0,24.0,13.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,0.0,2,2018,8,0.500000,0.866025,0.974928,-0.222521,1.0,6.123234e-17
3,1.0,Puerta del Sol A,0.0,0.0,24.0,17.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,0.0,2,2018,8,0.707107,0.707107,0.974928,-0.222521,1.0,6.123234e-17
4,1.0,Puerta del Sol A,0.0,0.0,24.0,22.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,0.0,2,2018,8,0.866025,0.500000,0.974928,-0.222521,1.0,6.123234e-17


In [15]:
temp.drop(columns=["name","longitude","address","month","time","weekday"])
temp.head(3)

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,...,holidays,weekday,year,month,hour_sin,hour_cos,weekday_sin,weekday_cos,month_sin,month_cos
0,1.0,Puerta del Sol A,0.0,2.0,24.0,11.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,0.0,2,2018,8,0.000000,1.000000,0.974928,-0.222521,1.0,6.123234e-17
1,1.0,Puerta del Sol A,0.0,0.0,24.0,18.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,0.0,2,2018,8,0.258819,0.965926,0.974928,-0.222521,1.0,6.123234e-17
2,1.0,Puerta del Sol A,0.0,2.0,24.0,13.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,0.0,2,2018,8,0.500000,0.866025,0.974928,-0.222521,1.0,6.123234e-17


In [ ]:
temp.to_csv("filename.processed.csv")